<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-28 15:19:56
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  5.37 L
Current:  1.32 C
-------------------
Today PnL: 3.69 K (0.03%)
Current PnL: -16.50 L (-11.5%)
CY Booked + Current PnL: -12.04 L (-8.39%)
-------------------
Total profit:  3.76 L
Total loss:  -20.27 L
-------------------
Total Booked + Current PnL: 14.94 L (12.82%)
Total Booked PnL: 31.44 L (26.98%)
Curr Year Booked PnL: 4.46 L (3.38%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 79.25 L (60.08%)
-------------------
Est LTT:  2.72 C
Est LTT PnL: 1.40 C (106.51%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 8.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LICI,785.05,983.00,-5.02,H-LC,92.47,233641.0,38949.0,10140.0,8.13,20.01,4.34,25.21,29.0,3.84,1.85,29.20,XY25,NTT,INSURANCE
62,RELIANCE,1291.83,1526.00,4.23,X-LC,47.31,155009.0,14200.0,11331.0,-0.89,10.08,7.31,18.13,17.0,1.25,1.22,22.39,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,89.25,100110.0,21748.0,12283.0,0.41,27.75,12.27,43.43,71.0,1.77,0.79,33.90,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,7.98,H-SC,86.02,159154.0,21294.0,16472.0,-0.41,15.45,10.35,27.39,118.0,1.29,1.26,37.94,X40,ATH,FMCG
56,PIDILITIND,2504.06,3576.00,-18.89,X-LC,19.35,89886.0,14764.0,17393.0,-1.04,19.65,19.35,42.81,11.0,0.85,0.71,12.73,X40,BTT,CHEMICALS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LICI,785.05,983.0,-5.02,H-LC,92.47,233641.0,38949.0,10140.0,8.13,20.01,4.34,25.21,29.0,3.84,1.85,29.20,XY25,NTT,INSURANCE
87,VALIANTORG,512.64,838.0,-367.00,H-SC,98.92,113464.0,-19822.0,104421.0,4.99,-14.87,92.03,63.47,131.0,-0.19,0.90,90.19,XR,NTT,CHEMICALS
75,SURYODAY,189.78,240.0,53.51,H-SC,63.44,109534.0,-40392.0,80069.0,4.58,-26.94,73.10,26.46,129.0,-0.50,0.87,40.32,XR,NTT,BANKS
16,CAMPUS,294.86,393.0,NaN,NaN,93.55,163030.0,-4745.0,60582.0,4.16,-2.83,37.16,33.28,175.0,-0.08,1.29,26.22,XY24,NTT,FOOTWEAR
20,COFFEEDAY,59.14,80.0,-55.73,L-SC,66.67,64512.0,-49037.0,89091.0,3.00,-43.19,138.10,35.27,238.0,-0.55,0.51,57.89,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,1149.24,-11.32,X-SC,18.28,81056.0,-19721.0,69497.0,-4.20,-19.57,85.74,49.39,65.0,-0.28,0.64,5.48,X40N,ATH,DURABLES
81,TITAGARH,1097.23,1548.00,-9.46,H-SC,90.32,158961.0,-33054.0,111940.0,-3.26,-17.21,70.42,41.08,127.0,-0.30,1.26,34.54,XY24,NTT,ENGINEERING
91,WHIRLPOOL,1167.49,2270.00,-38.45,M-SC,43.01,179443.0,7822.0,154249.0,-2.32,4.56,85.96,94.43,174.0,0.05,1.42,32.44,X40,NTT,DURABLES
58,QUESS,711.66,986.00,NaN,NaN,7.53,70846.0,-85008.0,145086.0,-2.28,-54.54,204.79,38.55,173.0,-0.59,0.56,8.48,XY24,NTT,MISC
42,IRCTC,904.87,1269.00,-26.56,M-MC,39.78,120986.0,-20174.0,76983.0,-2.22,-14.29,63.63,40.24,150.0,-0.26,0.96,15.59,XY24,BTT,TRAVEL


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-12.42,X-LC,54.84,249646.0,19047.0,112091.0,-0.66,8.26,44.90,56.86,1.0,0.17,1.97,13.94,X40,BTT,IT
80,TCS,3841.06,4998.00,-16.16,X-LC,40.86,241390.0,-23643.0,103484.0,-0.01,-8.92,42.87,30.12,2.0,-0.23,1.91,8.26,X40,BTT,IT
50,LTIM,5564.16,7197.33,-4.53,X-LC,84.95,183600.0,-16710.0,75496.0,-0.15,-8.34,41.12,29.35,4.0,-0.22,1.45,26.84,X200,ATH,IT
71,SIEMENS,4406.90,7969.85,-7.94,X-LC,83.87,83358.0,-26814.0,115893.0,1.07,-24.34,139.03,80.85,5.0,-0.23,0.66,22.77,AR,ATH,ELECTRICAL
32,HAVELLS,1593.67,2069.16,-6.46,X-LC,9.68,210623.0,-6116.0,70790.0,-1.53,-2.82,33.61,29.84,6.0,-0.09,1.66,10.49,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,9.80,H-SC,100.00,126983.0,29336.0,18336.0,1.91,30.04,14.44,48.82,138.0,1.60,1.00,41.07,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,8.17,H-MC,61.29,102748.0,17647.0,25451.0,1.34,20.74,24.77,50.65,92.0,0.69,0.81,29.16,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,89.25,100110.0,21748.0,12283.0,0.41,27.75,12.27,43.43,71.0,1.77,0.79,33.90,XY24,NTT,MISC
49,LICI,785.05,983.00,-5.02,H-LC,92.47,233641.0,38949.0,10140.0,8.13,20.01,4.34,25.21,29.0,3.84,1.85,29.20,XY25,NTT,INSURANCE


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,39.47,M-SC,65.59,144580.0,7002.0,87413.0,-0.29,5.09,60.46,68.62,182.0,0.08,1.14,25.34,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,9.80,H-SC,100.00,126983.0,29336.0,18336.0,1.91,30.04,14.44,48.82,138.0,1.60,1.00,41.07,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,8.17,H-MC,61.29,102748.0,17647.0,25451.0,1.34,20.74,24.77,50.65,92.0,0.69,0.81,29.16,X40N,ATH,CABLES
28,GILLETTE,8109.44,10330.69,7.98,H-SC,86.02,159154.0,21294.0,16472.0,-0.41,15.45,10.35,27.39,118.0,1.29,1.26,37.94,X40,ATH,FMCG
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,89.25,100110.0,21748.0,12283.0,0.41,27.75,12.27,43.43,71.0,1.77,0.79,33.90,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-0.89,H-SC,35.48,111527.0,9523.0,35499.0,0.17,9.34,31.83,44.14,111.0,0.27,0.88,27.09,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-30.66,M-MC,37.63,136503.0,7351.0,79718.0,2.77,5.69,58.40,67.42,155.0,0.09,1.08,32.76,XR,NTT,BANKS
33,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,73.12,152236.0,1323.0,59128.0,0.26,0.88,38.84,40.06,89.0,0.02,1.20,24.52,AR,ATH,AUTO
92,WIPRO,243.46,420.00,-10.62,M-LC,45.16,153400.0,2455.0,106996.0,-0.33,1.63,69.75,72.51,69.0,0.02,1.21,15.50,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.77,H-SC,48.39,122833.0,-503.0,137475.0,0.44,-0.41,111.92,111.05,108.0,-0.00,0.97,20.35,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-55.73,L-SC,66.67,64512.0,-49037.0,89091.0,3.00,-43.19,138.10,35.27,238.0,-0.55,0.51,57.89,XR,NTT,HOTELS
52,MASFIN,326.60,399.5,-22.76,H-SC,59.14,87525.0,-10455.0,32323.0,-0.29,-10.67,36.93,22.32,130.0,-0.32,0.69,26.94,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,-0.04,M-LC,89.25,100110.0,21748.0,12283.0,0.41,27.75,12.27,43.43,71.0,1.77,0.79,33.90,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-367.00,H-SC,98.92,113464.0,-19822.0,104421.0,4.99,-14.87,92.03,63.47,131.0,-0.19,0.90,90.19,XR,NTT,CHEMICALS
84,UJJIVANSFB,52.77,60.0,37.69,M-SC,36.56,119853.0,-22626.0,42152.0,-1.55,-15.88,35.17,13.70,217.0,-0.54,0.95,41.46,X40N,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,9.80,H-SC,100.00,126983.0,29336.0,18336.0,1.91,30.04,14.44,48.82,138.0,1.60,1.00,41.07,X40,ATH,PHARMA
87,VALIANTORG,512.64,838.00,-367.00,H-SC,98.92,113464.0,-19822.0,104421.0,4.99,-14.87,92.03,63.47,131.0,-0.19,0.90,90.19,XR,NTT,CHEMICALS
77,TANLA,917.30,1963.11,-34.99,H-SC,97.85,159249.0,-80166.0,353119.0,2.43,-33.48,221.74,114.01,110.0,-0.23,1.26,46.06,AR,ATH,IT
7,ASIANTILES,75.41,137.00,5933.33,M-SC,96.77,66206.0,-27604.0,104221.0,-0.04,-29.43,157.42,81.67,203.0,-0.26,0.52,28.21,XR,NTT,CERAMICS
74,STARHEALTH,551.56,761.00,5.88,M-SC,95.70,172970.0,-27246.0,103280.0,2.31,-13.61,59.71,37.97,194.0,-0.26,1.37,38.88,XY24,NTT,INSURANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.52
1,25,40.53
2,50,69.87


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.48
XY25    14.51
X40     14.27
X40N    13.69
AR      11.87
XR      10.60
X200     1.45
SR       1.15
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.69
M-SC    18.13
X-LC    17.38
H-MC     8.79
M-LC     7.85
H-LC     6.63
X-MC     5.71
M-MC     5.66
X-SC     2.82
L-SC     2.35
L-LC     1.16
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.18,-0.85,32.36
IT,10.11,-17.16,81.20
BANKS,9.81,-13.15,65.17
MISC,6.51,-31.17,89.19
FINANCE,6.04,-25.10,84.50
INSURANCE,5.48,3.62,25.27
PAINTS,5.47,-18.74,53.88
ELECTRICAL,4.83,-4.78,56.95
AUTO,4.51,-9.57,57.43


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2864794.0,26
AR,1235436.0,12
X40N,1218222.0,17
XR,1111160.0,14
X40,754605.0,11
XY25,466336.0,10
SR,199227.0,2
X200,75496.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2272679.0,21
M-SC,2024380.0,21
X-LC,780062.0,12
M-MC,553033.0,5
H-MC,514587.0,8
X-SC,352751.0,4
M-LC,313051.0,6
X-MC,310097.0,5
L-SC,300228.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      933924.0      6
M-SC       XY24      739896.0      6
H-SC       AR        549956.0      3
           X40N      501679.0      6
M-SC       X40N      472035.0      6
X-LC       X40       436171.0      5
M-MC       XY24      391556.0      3
M-SC       XR        289953.0      4
H-SC       XR        252312.0      4
H-LC       AR        246433.0      3
M-SC       X40       222237.0      2
M-LC       XY24      188410.0      4
H-MC       XY24      181115.0      2
L-SC       XR        152531.0      2
X-LC       AR        141594.0      2
X-MC       XY24      139713.0      2
X-LC       XY25      126801.0      4
M-SC       XY25      122217.0      1
X-SC       X40N      121977.0      2
           XR        116208.0      1
           SR        114566.0      1
H-MC       XR        113442.0      1
M-LC       XR        106996.0      1
X-MC       X40N       97080.0      2
M-SC       AR         93381.0      1
           SR         84661.0      1
L-SC       XY24       84512.0      1
M-MC       AR         81759.0      1
           XR         79718.0      1
X-LC       X200       75496.0      1
H-MC       XY25       74062.0      1
X-MC       XY25       73304.0      1
L-SC       AR         63185.0      1
H-MC       X40        61389.0      2
           AR         59128.0      1
L-LC       XY25       42167.0      1
H-SC       X40        34808.0      2
H-MC       X40N       25451.0      1
M-LC       XY25       17645.0      1
H-LC       XY25       10140.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
